# ETL

## Importación de librerias

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import gzip
import json
from textblob import TextBlob

## 1. Dataset 'steam_games.json. gz'

### Importación de datos del dataset

In [3]:
# Importación del Dataset
# Reemplaza 'mi_archivo.json.gz' con la ruta de tu archivo
with gzip.open('Dataset_inicial/steam_games.json.gz', mode='rt') as f:
    data = [json.loads(line) for line in f]

# Ponemos la variable como un DataFrame
games= pd.DataFrame(data)

### Muestra de los datos

In [17]:
games.head()

,publisher,genres,app_name,release_date,price,id,developer,average price
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,3.473871
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,25.29,643980,Secret Level SRL,25.29
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域,6.607647
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,7.823333
5,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,2018-01-04,9.99,774276,Poppermost Productions,26.79


In [5]:
games.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [6]:
# Cantidad de nulos por columnas
games.isnull().mean()*100

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

Se deduce que gran cantidad de los datos nulos del DataFrame afectaria considerablemente, sin embargo para esta ocasión se realizará la eliminación de los elementos nulos de manera sistematica

In [7]:
# Se eliminan las filas que no ofrecen información alguna
games.dropna(how= 'all', inplace= True)

# Registro num 88384 y 90890, en la mayoría no hay datos, podemos 
# eliminar estas filas
games.drop([88384,90890], inplace= True)

# Se puede eliminar la columna title por ser la misma a la 'app_name'
games.drop(columns= ['title'], inplace= True)

# Eliminamos las columnas que no ayudan a nuestro EDA
games.drop(columns= ['url', 'tags', 'reviews_url', 'early_access',
                     'specs'], inplace= True)


# Podemos realizar lo mismo para 'publisher' y 'developer'
condicion= games['publisher'].isnull() & games['developer'].notnull()
condicion2= games['publisher'].notnull() & games['developer'].isnull()

# Se reemplazan los valores nulos de las empresas que publicaron
# a desarrolladoras y viceversa
games.loc[condicion, 'publisher'] = games.loc[condicion, 'developer']
games.loc[condicion2, 'developer'] = games.loc[condicion2, 'publisher']

In [10]:
games.isnull().mean()*100

publisher       0.0
genres          0.0
app_name        0.0
release_date    0.0
price           0.0
id              0.0
developer       0.0
dtype: float64

### Imputación de datos

Realizaremos una imputación de los valores nulos de los precios con los precios promedios de los generos que se asemejan

In [9]:
# Terminamos de eliminar el resto de elementos nulos
games.dropna(inplace=True)

In [12]:
# Se convierte la columna 'genres' para que sea hashable
games['genres']= games['genres'].apply(lambda x: tuple(x) if x is not None else x)

In [14]:
# La variable prom tendra las columnas genero y precios
prom= games.loc[:,['genres','price']]

# ponemos las variables que estan como lista a tuples para hashearlas
prom['genres']= prom['genres'].apply(lambda x: tuple(x))

# Elegimos todas los registros que tienen como variable float en la columna 'Price'
prom= prom.loc[prom['price'].apply(lambda x: not isinstance(x, str)), :]

# Se realiza la agrupación de los generos y se estima el promedio
prom['price'].astype(float)
promedio= prom.groupby('genres')['price'].mean()

# saca pone los indices en forma de columna
promedio= promedio.reset_index()
# promedio['genres'] = promedio.genres.apply(lambda x: list(x))
promedio.columns = ['genres','average price']

In [15]:
# Luego de realizar la imputación de los datos realizamos la conexion entre la tabla games y promedio y eliminamos los datos None
games= pd.merge(games, promedio, on= 'genres', how= 'left')
games['price']= games['price'].fillna(games['average price'])

# Reemplazamos los valores que son 'str' con los valores que sacamos de los promedios de la columna 'average price'
games['price'] = games.apply(lambda row: row['average price'] if isinstance(row['price'], str) else row['price'], axis=1)
games.dropna(inplace= True)

In [16]:
games['genres']= games['genres'].apply(lambda x: list(x))

### Exportación de datos

In [20]:
# Exportamos el DataFrame a archivo .parquet
games.to_parquet('Dataset_ETL/steam_games.parquet', engine='pyarrow')

## 2. Dataset 'user_reviews.json.gz'

### Importación los datos del dataset

In [4]:
# se crea una nueva lista en blanco para anexar los datos a esa lista
rows=[]

# con el metodo gzip, se lee el archivo .gz
with gzip.open('Dataset_inicial/user_reviews.json.gz', 'rb') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line.decode(encoding= 'utf-8')))

# lo ponemos en un DataFrame llamado "reviews"
reviews= pd.DataFrame(rows)

### Exploración de los datos

In [11]:
reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


Se ve que la columna 'reviews' tiene anidados datos en formato JSON, realizaremos su segmentación

In [25]:
df_review_explot

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [26]:
df_norm

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59328,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,,Posted July 20.,,730,No ratings yet,True,:D


In [6]:
# Descomponemos los datos estructurados de la lista de la columna 'reviews'
df_review_explot = reviews.explode('reviews')
# Normalizamos los datos en formato json y lo guardamos en un DataFrame
df_norm = pd.json_normalize(df_review_explot['reviews'])

In [7]:
# Iniciamos los indices desde cero
df_review_explot.reset_index(drop=True, inplace=True)
df_norm.reset_index(drop=True, inplace=True)

In [8]:
# Concatenamos los DataFrames
reviews = pd.concat([df_review_explot,df_norm], axis=1)

In [10]:
# Eliminamos la columna
reviews.drop(columns=['reviews'],inplace=True)

Cantidad de nulos en el DataFrame

In [12]:
reviews.isnull().mean()*100

user_id        0.000000
user_url       0.000000
funny          0.047191
posted         0.047191
last_edited    0.047191
item_id        0.047191
helpful        0.047191
recommend      0.047191
review         0.047191
dtype: float64

In [13]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  object
 8   review       59305 non-null  object
dtypes: object(9)
memory usage: 4.1+ MB


- Se realiza el borrado de todos los nulos por no ser tantos
- Eliminamos las columnas 'funny','last_edited','user_url','posted','helpful' por no contener datos relevantes para el analisis

In [14]:
# Se realiza el borrado de todos los nulos por no ser tantos
reviews.dropna(inplace=True)
reviews.reset_index(drop=True, inplace=True)

In [15]:
# Eliminamos las columnas 'funny','last_edited','user_url','posted','helpful' por no contener datos relevantes para el analisis
reviews.drop(columns=['funny','last_edited','user_url','posted','helpful'],inplace=True)

### Análisis de sentimientos

In [17]:
# Funcion para el analisis de sentimiento usando la libreria Textblob
def analyze_sentiment(review):
    if pd.isna(review) or review.strip() == '':
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.1:
        return 0
    elif polarity > 0.1:
        return 2
    else:
        return 1

In [18]:
# Aplicar la función de análisis de sentimiento
reviews['sentiment_analysis'] = reviews['review'].apply(analyze_sentiment)

In [20]:
# Reemplazar la columna 'review' con 'sentiment_analysis'
reviews = reviews.drop(columns=['review'])

In [21]:
# Mostrar el DataFrame resultante
print(reviews)

                 user_id item_id recommend  sentiment_analysis
0      76561197970982479    1250      True                   2
1      76561197970982479   22200      True                   2
2      76561197970982479   43110      True                   1
3                js41637  251610      True                   2
4                js41637  227300      True                   1
...                  ...     ...       ...                 ...
59300  76561198312638244      70      True                   2
59301  76561198312638244  362890      True                   2
59302        LydiaMorley  273110      True                   1
59303        LydiaMorley     730      True                   2
59304        LydiaMorley     440      True                   2

[59305 rows x 4 columns]


### Exportación de datos

In [48]:
reviews.to_parquet('Dataset_ETL/user_reviews.parquet',engine='pyarrow')

## 3. Dataset 'users_items.json.gz'

### Importación del dataset

In [23]:
rows=[]

with gzip.open('Dataset_inicial/users_items.json.gz', 'rb') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line.decode(encoding= 'utf-8')))

items= pd.DataFrame(rows)



### Transformación, exploración y análisis del dataset

In [25]:
# Nomralizacion de la columna tipo JSON
df_items_explode = items.explode('items')
df_item_norm = pd.json_normalize(df_items_explode['items'])

In [26]:
# Inicializacion de indices
df_items_explode.reset_index(drop=True, inplace=True)
df_item_norm.reset_index(drop=True, inplace=True)

In [27]:
# Concatenizar los datasets
items = pd.concat([df_items_explode,df_item_norm], axis=1)

In [29]:
# Eliminamos la columna items
items.drop(columns=['items'], inplace=True)

In [41]:
# Normalizamos la columna item_name
items['item_name'] = items.item_name.str.title()

In [46]:
items.head()

,user_id,items_count,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,30,Day Of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,50,Half-Life: Opposing Force,0.0,0.0


In [36]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
 8   name_norm         object 
dtypes: float64(2), int64(1), object(6)
memory usage: 355.0+ MB


In [37]:
# Cantidad de nulos por columnas
items.isnull().sum()

user_id                 0
items_count             0
steam_id                0
user_url                0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
name_norm           16806
dtype: int64

In [38]:
condicion = items.playtime_forever.isnull()
items[condicion]

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,name_norm
3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN,NaN


- Se ve que hay una cantidad pequeña de nulos en comparación del tamaño del dataset, se procederá a eliminar los nulos.
- Eliminamos las columnas que no seran relevantes para nuestro analisis en este caso serán 'user_url'

In [39]:
# eliminamos nulos, columna innecesaria y reseteamos los indices
items.dropna(inplace=True)
items.drop(columns=['user_url'],inplace=True)
items.reset_index(drop=True, inplace=True)

In [45]:
# Vemos si la columna 'user_id' podria ser la misma que 'steam_id' para eliminar alguna de ellas
(items.user_id == items.steam_id).value_counts()

# Pero no son iguales

False    2950877
True     2202332
Name: count, dtype: int64

In [42]:
items.shape

(5153209, 7)

### Exportación del dataset

In [49]:
items.to_parquet('Dataset_ETL/users_items.parquet', engine='pyarrow')